In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from altaipony.fakeflares import aflare

import os
CWD = "/".join(os.getcwd().split("/")[:-2])

import pytest

from funcs.model import daylength, on_off, lambert, great_circle_distance, dot_ensemble_circular, full_model
from funcs.helper import no_nan_inf
import matplotlib 
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 


import astropy.units as u
from astropy.constants import R_sun

### Plots 

In [ ]:
# plot a grid

#latitudes, longitudes = dot_ensemble(np.pi/2-np.pi/8,np.pi/2-np.pi/10,4,num_pts=1e6)
latitudes, longitudes, pos = dot_ensemble_circular(25/180*np.pi,np.pi/2-np.pi/8,3,num_pts=300)
plt.figure(figsize=(8,8))
plt.scatter(longitudes/np.pi*180, latitudes/np.pi*180)
#plt.scatter([180/2-180/10],[180/2-180/8],)
plt.xlabel("longitude [deg]")
plt.ylabel("latitude [deg]")
#plt.savefig(f"{CWD}/analysis/plots/model/dot_ensemble.png",dpi=300)

In [ ]:
# Plot a model consisting of an ensemble of rotating flaring points on a sphere

I = 65.5 * np.pi / 180.
phi = np.linspace(0,6*np.pi,2000)
ampl = 20
fwhm = 3.
flare = aflare(phi, 2.9, fwhm, ampl,)

lamb, onoff, m = model(phi, latitudes, -longitudes, flare, I)

In [ ]:
plt.figure(figsize=(10,4))
for l,oo in zip(lamb,onoff):
    plt.plot(phi/2/np.pi, l*oo*flare+1, alpha=.1, c="grey")
plt.plot(phi/2/np.pi,m+1,c="k",linewidth=4, label="model light curve")#,\naverage of individual\ngrid dots (in grey)")
plt.plot(phi/2/np.pi,flare+1, label="underlying flare",c="r",linestyle="dashed",linewidth=4)
plt.xlabel("time [rotation periods]",fontsize=20)
plt.ylabel("rel. flux",fontsize=20)
plt.xlim(0,3)
plt.legend(fontsize=20)
#plt.yscale("log")
plt.tight_layout()
#plt.title(f"Flare with rel. amplitude={ampl}, and FWHM={fwhm} rotation periods",fontsize=20)
#plt.savefig(f"{CWD}/analysis/plots/model/rotating_flaring_ensemble_lightcurve.png",dpi=300);


In [ ]:
# create a heatmap of an ensemble of rotating flaring points on a sphere (need the cell above)
from matplotlib.colors import LogNorm
plt.figure(figsize=(15,10))
plt.imshow((lamb*onoff*flare)[:,:], norm=LogNorm(),aspect="auto")
plt.colorbar(label="flux")
plt.xlabel("time",fontsize=15)
plt.ylabel("index of grid dot",fontsize=15)
#plt.savefig(f"{CWD}/analysis/plots/model/rotating_flaring_ensemble_heatmap.png",dpi=300)

In [ ]:
# 10% percent area spot on the stellar surface
percent = 10
num_pts=1e4
indices = np.arange(0, num_pts, dtype=float) + 0.5

phi = np.arccos(1 - 2*indices/num_pts) #latitude
theta = np.pi * (1 + 5**0.5) * indices  #longitude

a = np.where(great_circle_distance(np.pi/3,- np.pi/4, np.pi/2 - phi%(2*np.pi), theta%(np.pi*2))<(percent/180*np.pi))[0]
phi2 = phi[a]
theta2 = theta[a]

x, y, z = np.cos(theta) * np.sin(phi), np.sin(theta) * np.sin(phi), np.cos(phi);
x2, y2, z2 = np.cos(theta2) * np.sin(phi2), np.sin(theta2) * np.sin(phi2), np.cos(phi2);
ax = plt.figure(figsize=(10,10)).add_subplot(111, projection='3d')
ax.scatter(x, y, z, s=.1);
ax.scatter(x2, y2, z2, s=3.5, c="k");
#plt.savefig(f"{CWD}/analysis/plots/model/10percent_spot_on_a_sphere.png",dpi=300)

In [ ]:
# plot a rotaionally modulated dot-flare
flare = aflare(phi, 1.5, 1.3, 1,)
plt.figure(figsize=(8,16))
l = 80/180*np.pi
I = np.arange(30,91,10)/180*np.pi
for i in I:
    D = daylength(l,i)
    modulation = lambert(phi,i,l)
    #plt.plot(phi/2/np.pi,modulation*on_off(D,phi),label=f"i={i/np.pi*180:.0f}" + r"$^{\circ}$")
    plt.plot(phi/2/np.pi,modulation*on_off(phi,D)*flare + i,label=f"i={i/np.pi*180:.0f}" + r"$^{\circ}$",
            linewidth=2,)
    plt.plot(phi/2/np.pi,flare + i,c="grey",linestyle="dotted")
plt.title(r"bright flaring dot at latitude 80$^{\circ}$ ",fontsize=15)
plt.xlabel("time [rotation periods]",fontsize=15)
plt.ylabel("flux",fontsize=15)
plt.legend(loc=(1.1,0),fontsize=15)
plt.xlim(0,3)
plt.tight_layout()
#plt.savefig(f"{CWD}/analysis/plots/model/rotating_bright_dot_flare.png",dpi=300)

In [ ]:
# plot visibility as a function of day time
plt.figure(figsize=(12,4))
phi = np.linspace(0,6*np.pi,500) #(longitude)
daylengths = np.arange(.1,.91,.2)
for d in daylengths:
    plt.plot(phi/2/np.pi, on_off(phi,d), linewidth=d*2,c="k", label=f"day length / Prot={d:.1f}")
plt.legend(loc=(1.05,0))
plt.xlabel("time [days]")
plt.ylabel("visibility")
plt.tight_layout()
#plt.savefig(f"{CWD}/analysis/plots/model/visibility_vs_day.png",dpi=300)

In [ ]:
# Plot daylengths:

I = np.linspace(0,np.pi/2,10)
L = np.linspace(-np.pi/2,np.pi/2, 500)[::-1]
plt.figure(figsize=(10,6))

    
for i in I:
    dl = []
    for l in L:
        dl.append(daylength(l,i))
    plt.plot(L/np.pi*180,dl,label=f"i={i/np.pi*180:.1f}")
plt.xlabel("latitude ",fontsize=15)
plt.ylabel("daylength",fontsize=15)
plt.ylim(-.1,1.1)
plt.xlim(-90,90)
plt.legend(loc=(1.1,0),fontsize=15)
plt.tight_layout()
#plt.savefig(f"{CWD}/analysis/plots/model/daylength.png",dpi=300)

In [4]:
df = pd.read_csv("/home/ekaterina/Documents/001_science/MalachiteMountains/data/summary/lcsi.csv")
inits = pd.read_csv("/home/ekaterina/Documents/001_science/MalachiteMountains/data/summary/inits.csv")


In [ ]:
for index in [4]:#range(5):
    row = df.loc[index,]

    I = row.i_mu
    ID = int(row.ID)#80 * np.pi / 180.


    valinits = inits[inits.ID ==str(ID)].iloc[-1]
    Fth = valinits.Fth
    R = valinits.R_Rsun * R_sun
    qlum = valinits.qlum_erg_s


    phi = np.linspace(0,6*np.pi,200)
    ampl = 1
    fwhm = 3
    theta = 70 * np.pi / 180.
    phi_a = np.pi
    fig, axes = plt.subplots(nrows=10,ncols=10, figsize=(10,10),)
    AX = [x for ax in axes for x in ax]
    for x, theta in zip(range(10),np.linspace(-I+.1,np.pi/2-.1,10)):
        for y, phi0 in zip(range(10),np.linspace(0,2*np.pi,10)):
            m = full_model(phi_a, theta, ampl, fwhm, I, phi0=phi0,
                          phi=phi, num_pts=100, qlum=qlum*u.erg/u.s,
                          Fth=Fth*u.erg/u.s/(u.cm**2), R=R, median=10)
            AX[10*x + y].plot(phi, m,)
            AX[10*x + y].set_xticks([])
            AX[10*x + y].set_yticks([])
            AX[10*x + y].text(0.81, 0.75, f"{theta/np.pi*180:.0f}, {phi0/np.pi*.5:.2f}",
                verticalalignment='bottom', horizontalalignment='right',
                transform=AX[10*x + y].transAxes,
                color='green', fontsize=12)
            AX[10*x + y].set_ylim(9.5,25)
            AX[10*x + y].set_xlim(phi[0],phi[-1])
    plt.tight_layout()
    #AX[4] = plt.text
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=.0, hspace=.0)
    plt.savefig(f"/home/ekaterina/Documents/001_science/MalachiteMountains/analysis/plots/model/"
                f"{ID}_{I/np.pi*180:.0f}_deg_latitude_flare_compilation.png", dpi=300)

In [43]:
for index in range(5):
    row = df.loc[index,]

    I = row.i_mu
    ID = int(row.ID)#80 * np.pi / 180.


    valinits = inits[inits.ID ==str(ID)].iloc[-1]
    Fth = valinits.Fth
    R = valinits.R_Rsun * R_sun
    qlum = valinits.qlum_erg_s


    phi = np.linspace(0,6*np.pi,200)
    ampl = 1
    fwhm = 3
    theta = 70 * np.pi / 180.
    phi_a = np.pi
    s = 20
    Theta, Phi0, Maxamp = np.full(s**2,0),np.full(s**2,0),np.full(s**2,0)
    theta_phi0 = np.meshgrid(np.linspace(-I,np.pi/2,s),np.linspace(0,2*np.pi,s))
    positions = np.vstack(map(np.ravel, theta_phi0))
    L = np.array(list(map(lambda x: np.max(full_model(phi_a, x[0], ampl, fwhm, I, phi0=x[1],
                              phi=phi, num_pts=100, qlum=qlum*u.erg/u.s,
                              Fth=Fth*u.erg/u.s/(u.cm**2), R=R, median=10)),positions.T)))
    d = pd.DataFrame({"theta":positions[0],"phi0":positions[1],"maxamp":L,})
    plt.figure(figsize=(8,6))
    plt.scatter(d.theta/np.pi*180, (d.maxamp-10)/10., s=50, alpha=.3, label=f"$i = {I/np.pi*180:.0f}^\circ$")
    plt.xlim(-I/np.pi*180, 90)
    plt.ylim(0,1)
    plt.xlabel(r"$\theta_f$ [deg]", fontsize=14)
    plt.ylabel(r"$a$", fontsize=14)
    plt.tight_layout()
    plt.legend(loc=2, frameon=False)
    plt.savefig(f"/home/ekaterina/Documents/001_science/MalachiteMountains/analysis/plots/model/"
                f"{ID}_{I/np.pi*180:.0f}_deg_latitude_flare_compilation_scatter.png", dpi=300)
    plt.close()

<>:28: DeprecationWarning: invalid escape sequence \c
<>:28: DeprecationWarning: invalid escape sequence \c
<>:28: DeprecationWarning: invalid escape sequence \c
<ipython-input-43-e4cafd77f11d>:28: DeprecationWarning: invalid escape sequence \c
  plt.scatter(d.theta/np.pi*180, (d.maxamp-10)/10., s=50, alpha=.3, label=f"$i = {I/np.pi*180:.0f}^\circ$")
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [94]:
for index in range(5):
    row = df.loc[index,]

    I = row.i_mu
    ID = int(row.ID)#80 * np.pi / 180.


    valinits = inits[inits.ID ==str(ID)].iloc[-1]
    Fth = valinits.Fth
    R = valinits.R_Rsun * R_sun
    qlum = valinits.qlum_erg_s


    phi = np.linspace(0,6*np.pi,200)
    ampl = 1
    fwhm = 3
    theta = 70 * np.pi / 180.
    phi_a = np.pi
    s = 30
    Theta, Phi0, Maxamp = np.full(s**2,0),np.full(s**2,0),np.full(s**2,0)
    thetagrid = np.random.rand(s**2)*(np.pi/2 + I) - I
    phigrid = np.random.rand(s**2)*2*np.pi/2
    positions = np.array([thetagrid,phigrid])

    secL = np.array(list(map(lambda x: func(x),positions.T)))
    primL = np.array(list(map(lambda x: np.max(full_model(phi_a, x[0], ampl, fwhm, I, phi0=x[1],
                              phi=phi, num_pts=100, qlum=qlum*u.erg/u.s,
                              Fth=Fth*u.erg/u.s/(u.cm**2), R=R, median=10)),positions.T)))
    d = pd.DataFrame({"theta":positions[0],"phi0":positions[1],"maxamp":primL,"secondmax":secL})
    fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(6,6))

#     plt.scatter(d.theta/np.pi*180, (d.maxamp-10)/10., s=50, alpha=.3,
#                 label=f" prim. max. at $i ={I/np.pi*180:.0f}^\circ$", c="r")
#     plt.scatter(d.theta/np.pi*180, (d.secondmax-10)/10., s=50, alpha=.3,
#                 label=f"sec. max at $i ={I/np.pi*180:.0f}^\circ$", c="b")
    plt.scatter(d.theta/np.pi*180, (d.secondmax-10)/(d.maxamp-10), s=50, alpha=.3,
            label=f"sec./prim. max at $i ={I/np.pi*180:.0f}^\circ$", c="b")
    plt.xlim(-I/np.pi*180, 90)
    plt.ylim(0,1)
    plt.xlabel(r"$\theta_f$ [deg]", fontsize=14)
    plt.ylabel(r"$a$", fontsize=14)
    plt.tight_layout()
    plt.legend(loc=2, frameon=False)
    plt.savefig(f"/home/ekaterina/Documents/001_science/MalachiteMountains/analysis/plots/model/"
                f"{ID}_{I/np.pi*180:.0f}_deg_latitude_flare_compilation_scatter_ratio.png", dpi=300)
    plt.close()

<>:37: DeprecationWarning: invalid escape sequence \c
<>:37: DeprecationWarning: invalid escape sequence \c
<>:37: DeprecationWarning: invalid escape sequence \c
<ipython-input-94-6c3ad8a4155f>:37: DeprecationWarning: invalid escape sequence \c
  label=f"sec./prim. max at $i ={I/np.pi*180:.0f}^\circ$", c="b")


In [92]:
def func(x):
    m = full_model(phi_a, x[0], ampl, fwhm, I, phi0=x[1],
                              phi=phi, num_pts=100, qlum=qlum*u.erg/u.s,
                              Fth=Fth*u.erg/u.s/(u.cm**2), R=R, median=10)
    peaks = find_peaks(m)[0]
    if len(peaks) > 1:
        return np.sort(m[peaks])[-2]
    else: 
        return 10

In [80]:
from scipy.signal import find_peaks
import seaborn as sns

In [50]:
m[L[0][0]]

array([11.88144923, 10.50566567])